In [ ]:
pip install scholarly

In [ ]:
from scholarly import scholarly
from scholarly import ProxyGenerator
import os
import csv

In [ ]:
pg = ProxyGenerator()
scholarly.use_proxy(pg, pg)

In [ ]:
author_names = ['Wolf Moskovich']
author_ids = ['D1HB8BAAAAAJ']

In [ ]:
keys = ['author','title','pub_year','number_of_co_authors','co_authors']

output_file =  open('faculty_pubs.csv', 'a', newline='')
dict_writer = csv.DictWriter(output_file, keys)
if os.path.getsize('faculty_pubs.csv') == 0:
    dict_writer.writeheader()
    output_file.flush()
dict_reader = csv.DictReader(open('faculty_pubs.csv', newline=''), keys)
csv_reader = [row for row in dict_reader]



for name in author_names:
    #author = next(scholarly.search_author(name))
    author = scholarly.search_author_id('D1HB8BAAAAAJ')
    author =  scholarly.fill((author),sections = ['publications'])
    pubs = author['publications']
    author_pubs_in_file = [row for row in csv_reader if row['author'] == name]
    num_author_pubs_in_file = len(author_pubs_in_file)
    if num_author_pubs_in_file >= len(author['publications']): continue
    i = 0
    for pub in pubs:
        if i < num_author_pubs_in_file:
            i += 1
            continue
        pub = scholarly.fill(pub)
        if len([x for x in author_pubs_in_file if x['title'] == pub['bib']['title']]) > 0 : continue
        pub_res = {
            "author":name,
            "title":pub['bib']['title']
            }

        if 'author' in pub['bib']:
            pub_res['number_of_co_authors'] = len(pub['bib']['author'].split(' and ')) - 1
            pub_res['co_authors'] = pub['bib']['author']
        else:
            pub_res["number_of_co_authors"] = ''

        if 'pub_year' in pub['bib']:
            pub_res['pub_year'] = pub['bib']['pub_year']
        else:
            pub_res['pub_year'] = ''

        dict_writer.writerows([pub_res])
        output_file.flush()